# Better Performance with MKL BLAS on MXNet 1.6 Deep Learning Containers

Inference speed and performance is often times one of the most crucial factors for deciding to deploy a model in a production environment. Small increases in latency can be costly, so every bit of performance boost that can be had can help the overall costs. Because of our obessesion for customers, the MXNet team has brought a solution to improve the latency for inference using MXNet 1.6 Deep learning containers. In this post, we dicuss the improvement that was made to the MXNet 1.6.0 DLC version to make use of highly optimized matrix operators. The enhancement comes in the form of compiling MXNet with a dependency on Intel MKL BLAS instead of the default, oneDNN. As one will see, the performance boost can be up to 30% in latency reduction, making this a worthwhile option for our customers to implement in their production environment. To describe and show the enhancements in more detail, we will use the MNIST dataset to briefly give context of performing inference on Amazon SageMaker. Then, we will discuss the differences between the MKL BLAS and oneDNN libraries. Lastly, we will show how to implement the enhancement in your environment so that you can take advantage of the performance boost. 

**Note: MXNet 1.7+ Deep Learning Containers have this enhancement as a default, so this solution applies to customers who don't want to change the MXNet version from 1.6.0**

## Inference on Amazon SageMaker

Amazon SageMaker makes it really easy to deploy, host and maintain models. As part of that, choosing what framework or deep learning container to use is also a matter of setting a parameter. For the remainder of this post, we will use the MNIST dataset to quickly give examples for context and to show the performance difference between the MKL BLAS library and oneDNN library. But first, here is a brief example of how to perform inference in SageMaker.

### Setup

First we define a few variables that are needed to perform operations in SageMaker

In [84]:
!pip install gluonnlp --quiet
!pip install bert --quiet
!pip install mxnet --upgrade --quiet

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [85]:
!pip install gluoncv --quiet

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [86]:
from sagemaker import get_execution_role
from sagemaker.session import Session
import sagemaker
from sagemaker.mxnet.model import MXNetModel, MXNetPredictor
import pandas as pd
from sagemaker import utils
import boto3
import gzip
import os
import struct
import numpy as np
import tempfile
import mxnet as mx
import tarfile
import urllib.request
import gluonnlp as nlp
import cv2
from transform import BERTDatasetTransform
from gluonnlp.calibration import BertLayerCollector
from utils import test_bert, deploy_bert, deploy_model
# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = Session().default_bucket()

# Bucket location where results of model training are saved.
model_artifacts_location = 's3://{}/mxnet-mnist-example/artifacts'.format(bucket)

# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()
sagemaker_session = Session()
region = boto3.Session().region_name
test_data_location = 'sagemaker-sample-data-{}'.format(region)

# containers that will be used for comparision
oneDnn_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py36-ubuntu16.04-v3.7"
mkl_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py36-ubuntu16.04-v3.8"

# Dot operator using AWS Deep Learning Containers

### Setup Data 

This dataset contains 10,000 images that 28x28 pixels.

In [87]:
tweets = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1", names=["target", "ids", "date", "flag", "user", "text"])

### Create an inference Endpoint

We use the ``MXNet model`` object to load model data and deploy an ``MXNetPredictor``. This creates a Sagemaker **Endpoint** -- a hosted prediction service that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. Here we will deploy the model to a single ``ml.m4.xlarge`` instance. By not setting the ``image`` parameter, ``MXNetModel`` uses the default deep learning container image.

## MKL BLAS vs oneDNN

The MKL BLAS and oneDNN libraries of math routines that are used to perform mathmatical operations on data. You can think of these as low level instructions that progamming languages use to perform computations. In the case of MXNet, it uses these libraries to for its operations such as dot products and other computationally expensive operations. MKL BLAS as version implemented by Intel, that uses highly optimized operators for CPU. These operators like the ``dot`` operator, are much faster than the operators found on the default library, oneDNN. So in order to take advantage of the speed boost, the MXNet team packaged the the MXNet 1.6 version with Intel's MKL BLAS library in a deep learning container available for use. In the diagram below, you can see the changes that are made between the different math libraries and MXNet versions.  

### Performance

In order to see the performance increase, we describe now how to implement the MKL BLAS and oneDNN based deep learning containers using SageMaker. 

#### oneDNN 

Notice the parameter ``image`` is set to a uri. This is the deep learning container uri, and the ``v3.7`` specifies a version of this MXNet 1.6 container that is compiled with oneDNN. 

In [5]:
ecr_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py36-ubuntu16.04-v3.7"
onednn_predictor = deploy_bert(sagemaker_session, ecr_image, 'ml.m4.xlarge', "1.6.0", role)

Deploying...
-----------------!
Predicting...


In [9]:
times = inference(tweets.text[:100], onednn_predictor)

Avg: 55.409968686103824 Std: 0.12656422301418305 with 10 loops


[55.78212642669678,
 55.4226975440979,
 55.36242318153381,
 55.377811670303345,
 55.33304786682129,
 55.36034035682678,
 55.37596607208252,
 55.37017631530762,
 55.3291597366333,
 55.38593769073486]

#### MKL BLAS

To specify the deep learning container with MKL BLAS, you change the version identifier in the image uri to ``v3.8``.

In [10]:
ecr_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py36-ubuntu16.04-v3.8"
mkl_predictor = deploy_bert(sagemaker_session, ecr_image, 'ml.m4.xlarge', "1.6.0", role)

Deploying...
-----------------!
Predicting...


In [11]:
times = inference(tweets.text[:100], mkl_predictor)

Avg: 48.03284316062927 Std: 0.09309706624115292 with 10 loops


In [15]:
ecr_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py36-ubuntu16.04-v3.8"
mklq_predictor = deploy_bert("bert_sst_quantized.tar.gz", sagemaker_session, ecr_image, 'ml.m4.xlarge', "1.6.0", role, script="bert_inference_quantized.py")

Deploying...
-------------------!
Endpoint Name: bert-1605561706-78b3


In [17]:
times = inference(tweets.text[:100], mklq_predictor)

Avg: 59.52104277610779 Std: 0.1776845656988009 with 10 loops


# SSD

In [124]:
ecr_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-gpu-py36-cu101-ubuntu16.04-v3.7"
ssd_predictor = deploy_model("models/ssd_512_resnet50_v1_voc.tar.gz", "ssd-test", sagemaker_session, ecr_image, 'ml.p2.8xlarge', "1.6.0", role, script="ssd_inference-batch50.py")

Deploying...
-------------------!
Endpoint Name: ssd-test-1606158680-a1ac


In [125]:
#ssd_predictor = MXNetPredictor("ssd-test-1605820634-af4b")
img = cv2.imread("street_small.jpg")
times_ssd = inference(img, ssd_predictor)

Avg: 48.4411301612854 Std: 1.2594568489112994 with 10 loops


In [126]:
ecr_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-gpu-py36-cu101-ubuntu16.04-v3.8"
mkl_ssd_predictor = deploy_model("models/ssd_512_resnet50_v1_voc.tar.gz", "ssd-test-mkl", sagemaker_session, ecr_image, 'ml.p2.8xlarge', "1.6.0", role, script="ssd_inference-batch50.py")

Deploying...
------------------!
Endpoint Name: ssd-test-mkl-1606159796-52ae


In [128]:
times_ssd_mkl = inference(img, mkl_ssd_predictor)

Avg: 46.63021864891052 Std: 0.454994058507219 with 10 loops


In [ ]:
predictor.delete_endpoint()
oneDNN_predictor.delete_endpoint()
mklblas_predictor.delete_endpoint()